In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

pd.set_option("display.max_columns", None)
Path("data/raw").mkdir(parents=True, exist_ok=True)
Path("data/processed").mkdir(parents=True, exist_ok=True)

In [ ]:
import os
os.listdir()


['.config', 'data', 'sample_data']

In [ ]:
import pandas as pd
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(
    "Medicare_IP_Hospitals_by_Provider_and_Service_2023.csv",
    encoding="latin1"
)

df.head()


Saving Medicare_IP_Hospitals_by_Provider_and_Service_2023.csv to Medicare_IP_Hospitals_by_Provider_and_Service_2023.csv


,Rndrng_Prvdr_CCN,Rndrng_Prvdr_Org_Name,Rndrng_Prvdr_City,Rndrng_Prvdr_St,Rndrng_Prvdr_State_FIPS,Rndrng_Prvdr_Zip5,Rndrng_Prvdr_State_Abrvtn,Rndrng_Prvdr_RUCA,Rndrng_Prvdr_RUCA_Desc,DRG_Cd,DRG_Desc,Tot_Dschrgs,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,2.0,Metropolitan area high commuting: primary flow...,3,ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRIN...,14,663764.35714,120219.928570,115544.142860
1,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,2.0,Metropolitan area high commuting: primary flow...,23,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,26,180980.88462,37321.038462,35261.807692
2,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,2.0,Metropolitan area high commuting: primary flow...,24,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE ...,12,105824.33333,26936.666667,25048.916667
3,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,2.0,Metropolitan area high commuting: primary flow...,25,CRANIOTOMY AND ENDOVASCULAR INTRACRANIAL PROCE...,16,242539.50000,34745.375000,32438.625000
4,10001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,1,36301,AL,2.0,Metropolitan area high commuting: primary flow...,38,EXTRACRANIAL PROCEDURES WITH CC,11,122741.18182,14999.818182,9579.363636


Cleaning CMS data (currency, types, missing values


In [ ]:
df.columns


Index(['Rndrng_Prvdr_CCN', 'Rndrng_Prvdr_Org_Name', 'Rndrng_Prvdr_City',
       'Rndrng_Prvdr_St', 'Rndrng_Prvdr_State_FIPS', 'Rndrng_Prvdr_Zip5',
       'Rndrng_Prvdr_State_Abrvtn', 'Rndrng_Prvdr_RUCA',
       'Rndrng_Prvdr_RUCA_Desc', 'DRG_Cd', 'DRG_Desc', 'Tot_Dschrgs',
       'Avg_Submtd_Cvrd_Chrg', 'Avg_Tot_Pymt_Amt', 'Avg_Mdcr_Pymt_Amt'],
      dtype='object')

In [ ]:
currency_cols = ["Avg_Submtd_Cvrd_Chrg", "Avg_Tot_Pymt_Amt"]

for col in currency_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r"[$,]", "", regex=True)
        .astype(float)
    )

In [ ]:
df = df.dropna(subset=["Tot_Dschrgs", "Avg_Submtd_Cvrd_Chrg"])

In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146427 entries, 0 to 146426
Data columns (total 15 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Rndrng_Prvdr_CCN           146427 non-null  int64  
 1   Rndrng_Prvdr_Org_Name      146427 non-null  object 
 2   Rndrng_Prvdr_City          146427 non-null  object 
 3   Rndrng_Prvdr_St            146427 non-null  object 
 4   Rndrng_Prvdr_State_FIPS    146427 non-null  int64  
 5   Rndrng_Prvdr_Zip5          146427 non-null  int64  
 6   Rndrng_Prvdr_State_Abrvtn  146427 non-null  object 
 7   Rndrng_Prvdr_RUCA          146423 non-null  float64
 8   Rndrng_Prvdr_RUCA_Desc     146423 non-null  object 
 9   DRG_Cd                     146427 non-null  int64  
 10  DRG_Desc                   146427 non-null  object 
 11  Tot_Dschrgs                146427 non-null  int64  
 12  Avg_Submtd_Cvrd_Chrg       146427 non-null  float64
 13  Avg_Tot_Pymt_Amt           14

In [ ]:
df = df.rename(columns={
    "Tot_Dschrgs": "discharges",
    "Avg_Submtd_Cvrd_Chrg": "avg_covered_charges",
    "Avg_Tot_Pymt_Amt": "avg_total_payments",
    "Rndrng_Prvdr_State_Abrvtn": "state",
    "DRG_Desc": "drg"
})
#columns name too long lol

In [ ]:
df["discharges"].describe()


In [ ]:
before = df.shape[0]

df = df.dropna(subset=["discharges", "avg_covered_charges", "state", "drg"])
df = df[(df["discharges"] > 0) & (df["avg_covered_charges"] > 0)]

after = df.shape[0]
before, after, (before - after)


(146427, 146427, 0)

In [ ]:
df[["discharges", "avg_covered_charges", "avg_total_payments", "Avg_Mdcr_Pymt_Amt"]].describe()

,discharges,avg_covered_charges,avg_total_payments,Avg_Mdcr_Pymt_Amt
count,146427.000000,1.464270e+05,146427.000000,146427.000000
mean,33.875754,9.079423e+04,18512.962895,15331.359323
std,49.814354,1.200348e+05,21313.196903,18594.048969
min,11.000000,3.368500e+03,1938.428571,180.285714
25%,14.000000,3.526298e+04,8685.922619,6794.309873
50%,20.000000,5.866857e+04,12830.714286,10548.181818
75%,35.000000,1.042167e+05,20403.218750,16814.950605
max,3210.000000,1.041893e+07,761738.782610,751479.043480


due to RAM restriction created a D_small

In [ ]:
TOP_N_DRG = 30  # change to 20/30/50 depending on RAM

top_drgs = (
    df.groupby("drg")["discharges"]
    .sum()
    .sort_values(ascending=False)
    .head(TOP_N_DRG)
    .index
)

df_small = df[df["drg"].isin(top_drgs)].copy()
df_small.shape


(54568, 15)

In [ ]:
total_discharges_all = df["discharges"].sum()
total_discharges_small = df_small["discharges"].sum()

coverage = total_discharges_small / total_discharges_all
total_discharges_all, total_discharges_small, coverage


(np.int64(4960325), np.int64(2925516), np.float64(0.5897831291296437))

In [ ]:
clean_path = f"data/processed/cms_2023_clean.csv"
small_path = f"data/processed/cms_2023_top{TOP_N_DRG}_drgs.csv"

df.to_csv(clean_path, index=False)
df_small.to_csv(small_path, index=False)

clean_path, small_path


('data/processed/cms_2023_clean.csv', 'data/processed/cms_2023_top30_drgs.csv')

In [ ]:
from google.colab import files

files.download("data/processed/cms_2023_clean.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>